In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from skimage import transform
import pickle

In [2]:
def generate_tabular_dataset(srcpath,name):

    exercices = os.listdir(srcpath)


    if not os.path.exists(srcpath + '/' + name + '.csv') :
        data_train = pd.DataFrame(columns = ['NOISE_X', 
                                             'NOISE_Y',
                                             'LEFT_SHOULDER_X',
                                             'LEFT_SHOULDER_Y',
                                             'RIGHT_SHOULDER_X',
                                             'RIGHT_SHOULDER_Y',
                                             'LEFT_ELBOW_X',
                                             'LEFT_ELBOW_Y',
                                             'RIGHT_ELBOW_X',
                                             'RIGHT_ELBOW_Y',
                                             'LEFT_WRIST_X',
                                             'LEFT_WRIST_Y',
                                             'RIGHT_WRIST_X',
                                             'RIGHT_WRIST_Y',
                                             'LEFT_PINKY_X',
                                             'LEFT_PINKY_Y',
                                             'RIGHT_PINKY_X',
                                             'RIGHT_PINKY_Y',
                                             'LEFT_INDEX_X',
                                             'LEFT_INDEX_Y',
                                             'RIGHT_INDEX_X',
                                             'RIGHT_INDEX_Y',
                                             'LEFT_THUMB_X',
                                             'LEFT_THUMB_Y',
                                             'RIGHT_THUMB_X',
                                             'RIGHT_THUMB_Y',
                                             'LEFT_HIP_X',
                                             'LEFT_HIP_Y',
                                             'RIGHT_HIP_X',
                                             'RIGHT_HIP_Y',
                                             'LEFT_KNEE_X',
                                             'LEFT_KNEE_Y',
                                             'RIGHT_KNEE_X',
                                             'RIGHT_KNEE_Y',
                                             'LEFT_ANKLE_X',
                                             'LEFT_ANKLE_Y',
                                             'RIGHT_ANKLE_X',
                                             'RIGHT_ANKLE_Y',
                                             'LEFT_HEEL_X', 
                                             'LEFT_HEEL_Y',
                                             'RIGHT_HEEL_X', 
                                             'RIGHT_HEEL_Y',
                                             'LEFT_FOOT_INDEX_X', 
                                             'LEFT_FOOT_INDEX_Y',
                                             'RIGHT_FOOT_INDEX_X', 
                                             'RIGHT_FOOT_INDEX_Y',
                                             'PATH',
                                             'EXERCICE'])

    else : 
        data_train = pd.read_csv(srcpath + '/' + name + '.csv')

    
    for ex in exercices : 
        if 'csv' not in ex :
            images = os.listdir(srcpath + '/' + ex)
            print('Processing exercice ' + ex)
            if len(images):
                with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

                    for im in tqdm(images) :
                        path = srcpath + '/' + ex + '/' + im

                        try :
                        
                            if path not in data_train.PATH.unique() :

                                image = cv2.imread(path)
                                image.flags.writeable = False
                                flip_image = tf.image.flip_left_right(image).numpy()
                                to_process = {path : image,
                                              path + '_fliped' : flip_image}


                                for key in to_process.keys():

                                    # Make detection
                                    results = pose.process(to_process[key])
                                    landmarks = results.pose_landmarks.landmark


                                    new_entry = {'NOISE_X' : landmarks[mp_pose.PoseLandmark.NOSE.value].x, 
                                                'NOISE_Y' : landmarks[mp_pose.PoseLandmark.NOSE.value].y,
                                                'LEFT_SHOULDER_X' : landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                                'LEFT_SHOULDER_Y' : landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                                                'RIGHT_SHOULDER_X' : landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                                'RIGHT_SHOULDER_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                                                'LEFT_ELBOW_X' : landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                                'LEFT_ELBOW_Y' : landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                                                'RIGHT_ELBOW_X' : landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                                'RIGHT_ELBOW_Y': landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                                                'LEFT_WRIST_X' : landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                                'LEFT_WRIST_Y' : landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
                                                'RIGHT_WRIST_X' : landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                                'RIGHT_WRIST_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,
                                                'LEFT_PINKY_X' : landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].x,
                                                'LEFT_PINKY_Y' : landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].y,
                                                'RIGHT_PINKY_X' : landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].x,
                                                'RIGHT_PINKY_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].y,
                                                'LEFT_INDEX_X' : landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,
                                                'LEFT_INDEX_Y': landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y,
                                                'RIGHT_INDEX_X' : landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,
                                                'RIGHT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y,
                                                'LEFT_THUMB_X' : landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].x,
                                                'LEFT_THUMB_Y' : landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].y,
                                                'RIGHT_THUMB_X' : landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].x,
                                                'RIGHT_THUMB_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y,
                                                'LEFT_HIP_X' : landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                                'LEFT_HIP_Y' : landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                                                'RIGHT_HIP_X' : landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                                'RIGHT_HIP_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                                                'LEFT_KNEE_X' : landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                                'LEFT_KNEE_Y' : landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                                                'RIGHT_KNEE_X' : landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                                                'RIGHT_KNEE_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                                                'LEFT_ANKLE_X' : landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                                'LEFT_ANKLE_Y' : landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
                                                'RIGHT_ANKLE_X' : landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                                                'RIGHT_ANKLE_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
                                                'LEFT_HEEL_X' : landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                                                'LEFT_HEEL_Y' : landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y,
                                                'RIGHT_HEEL_X' : landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, 
                                                'RIGHT_HEEL_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y,
                                                'LEFT_FOOT_INDEX_X' : landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,  
                                                'LEFT_FOOT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y,  
                                                'RIGHT_FOOT_INDEX_X' : landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                                                'RIGHT_FOOT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y,
                                                'PATH' : key,
                                                'EXERCICE' : ex}

                                    data_train = data_train.append(new_entry,ignore_index=True)
                        except :
                            pass

    data_train.to_csv(srcpath + '/' + name + '.csv',index=False)
    return data_train




In [3]:
df_train = generate_tabular_dataset('./Data Train','train')

Processing exercice pullup


100%|██████████| 612/612 [00:14<00:00, 41.99it/s] 


Processing exercice pushup


100%|██████████| 683/683 [00:16<00:00, 40.30it/s] 


Processing exercice situp


100%|██████████| 959/959 [00:21<00:00, 45.50it/s] 


In [4]:
df_test = generate_tabular_dataset('./Data Test','test')

Processing exercice pullup


100%|██████████| 301/301 [00:07<00:00, 41.82it/s]


Processing exercice pushup


100%|██████████| 303/303 [00:07<00:00, 39.84it/s] 


Processing exercice situp


100%|██████████| 521/521 [00:10<00:00, 50.95it/s] 


In [5]:
X_cols = ['NOISE_X', 'NOISE_Y', 'LEFT_SHOULDER_X', 'LEFT_SHOULDER_Y',
       'RIGHT_SHOULDER_X', 'RIGHT_SHOULDER_Y', 'LEFT_ELBOW_X', 'LEFT_ELBOW_Y',
       'RIGHT_ELBOW_X', 'RIGHT_ELBOW_Y', 'LEFT_WRIST_X', 'LEFT_WRIST_Y',
       'RIGHT_WRIST_X', 'RIGHT_WRIST_Y', 'LEFT_PINKY_X', 'LEFT_PINKY_Y',
       'RIGHT_PINKY_X', 'RIGHT_PINKY_Y', 'LEFT_INDEX_X', 'LEFT_INDEX_Y',
       'RIGHT_INDEX_X', 'RIGHT_INDEX_Y', 'LEFT_THUMB_X', 'LEFT_THUMB_Y',
       'RIGHT_THUMB_X', 'RIGHT_THUMB_Y', 'LEFT_HIP_X', 'LEFT_HIP_Y',
       'RIGHT_HIP_X', 'RIGHT_HIP_Y', 'LEFT_KNEE_X', 'LEFT_KNEE_Y',
       'RIGHT_KNEE_X', 'RIGHT_KNEE_Y', 'LEFT_ANKLE_X', 'LEFT_ANKLE_Y',
       'RIGHT_ANKLE_X', 'RIGHT_ANKLE_Y', 'LEFT_HEEL_X', 'LEFT_HEEL_Y',
       'RIGHT_HEEL_X', 'RIGHT_HEEL_Y', 'LEFT_FOOT_INDEX_X',
       'LEFT_FOOT_INDEX_Y', 'RIGHT_FOOT_INDEX_X', 'RIGHT_FOOT_INDEX_Y']
       
target = 'EXERCICE'
encode = LabelEncoder()

In [6]:
X_train = df_train[X_cols].apply(lambda x : x - np.mean(x),axis=1)
y_train = encode.fit_transform(df_train[target])

In [7]:
X_test = df_test[X_cols].apply(lambda x : x - np.mean(x),axis=1)
y_test = encode.transform(df_test[target])

In [16]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(64,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(32,activation = 'relu'))
model.add(tf.keras.layers.Dense(3,activation='softmax'))

In [17]:
model.compile(loss = 'SparseCategoricalCrossentropy')

In [18]:
model.fit(X_train,y_train,epochs = 200,validation_data= (X_test,y_test))

Epoch 1/200
100/100 [==============================] - 1s 5ms/step - loss: 0.8784 - val_loss: 0.6719
Epoch 2/200
100/100 [==============================] - 0s 4ms/step - loss: 0.6390 - val_loss: 0.5470
Epoch 3/200
100/100 [==============================] - 0s 4ms/step - loss: 0.5493 - val_loss: 0.4556
Epoch 4/200
100/100 [==============================] - 0s 4ms/step - loss: 0.4987 - val_loss: 0.4183
Epoch 5/200
100/100 [==============================] - 0s 4ms/step - loss: 0.4621 - val_loss: 0.3887
Epoch 6/200
100/100 [==============================] - 0s 3ms/step - loss: 0.4515 - val_loss: 0.3726
Epoch 7/200
100/100 [==============================] - 0s 4ms/step - loss: 0.4180 - val_loss: 0.3413
Epoch 8/200
100/100 [==============================] - 0s 4ms/step - loss: 0.3931 - val_loss: 0.3373
Epoch 9/200
100/100 [==============================] - 0s 3ms/step - loss: 0.3722 - val_loss: 0.3200
Epoch 10/200
100/100 [==============================] - 0s 3ms/step - loss: 0.3842 - val_lo

In [19]:
proba_model = model.predict(X_test)
predictions = np.argmax(proba_model,axis=1)
metrics = tf.keras.metrics.Accuracy()

In [20]:
print('Accuracy : {}'.format(metrics(y_test,predictions)))

Accuracy : 0.9355653524398804


In [21]:
with open('./Model/encoder.pkl','wb') as file:
    pickle.dump(encode,file)

In [22]:
model.save('./Model/model')

INFO:tensorflow:Assets written to: ./Model/model\assets
